### Numerical weather prediction (NWP) example
Code to replicate the CPA plots to compare the performance of the HRES forecast and the persistence forecast for the variables 2m temperature, wind speed and precipitation for different leadtimes. As respective observation the ERA5 reanalysis product is taken. The data can be downloaded from the following sources:
- https://confluence.ecmwf.int/display/TIGGE
- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

Before the netCDF files were imported into Python they were modified by CDO.

The following code computes the CPA plot for precipitation. To obtain the respective plots for wind speed and Temperature replace the name of the variable.

In [ ]:
# Import packages
import xarray as xr 
import numpy as np
from netCDF4 import Dataset
from scipy.stats import rankdata

#### CPA between Persistence and Reanalysis 
The persistence forecast uses the present condition as forecast for future conditions. For this comparison the ERA5 reanalysis product is used

In [ ]:
target_obs = 'Precip_Reanalysis_2006-2018.nc'
DS_observation = xr.open_dataset(target_obs)

# set lead time: For Leadtime of one day ahead use lead=1 and for Leadtime of 5 days ahead use lead=5
lead = 1
# index
m = 0
# start dataset from 01.01.2006 
first_day = 5
# initialize
CPA_All = np.zeros(45)
CPAEurop = np.zeros((199,279))
# indices for reanalysis
indices_start = np.array([0,   90,  181,  273,  365,  456,  547,  639,  731,  821,  912,
       1004, 1096, 1186, 1277, 1369, 1461, 1551, 1642, 1734, 1826, 1917,
       2008, 2100, 2192, 2282, 2373, 2465, 2557, 2647, 2738, 2830, 2922,
       3012, 3103, 3195, 3287, 3378, 3469, 3561, 3653, 3743, 3834, 3926,
       4018])
indices_end = np.array([365,  456,  547,  639,  731,  821,  912, 1004, 1096, 1186, 1277,
       1369, 1461, 1551, 1642, 1734, 1826, 1917, 2008, 2100, 2192, 2282,
       2373, 2465, 2557, 2647, 2738, 2830, 2922, 3012, 3103, 3195, 3287,
       3378, 3469, 3561, 3653, 3743, 3834, 3926, 4018, 4108, 4199, 4291,
       4383])
indices_start = indices_start + first_day
indices_end = indices_end + first_day

for indx in range(0,45):
    start = indices_start[indx]
    end = indices_end[indx]

    predictor_nc = np.asarray(DS_observation.tp[(start-lead):(end-lead),:,:])
    response_nc = np.asarray(DS_observation.tp[start:end,:,:]) 
    
    response = response_nc.flatten()
    predictor = predictor_nc.flatten()
             
    resOrder = np.argsort(response)
    resSort = response[resOrder] 
    preSort = predictor[resOrder]                
    preRank = rankdata(preSort, method='average')
    resRank = rankdata(resSort, method='average')
    resClass = rankdata(resSort, method='dense')
    CPA_All[m] = (np.cov(resClass,preRank)[0][1]/np.cov(resClass,resRank)[0][1]+1)/2 
    m = m+1

np.savetxt('CPA_Precipitation_24.txt',CPA_All)

#### CPA between HRES and Reanalysis 

Hres from tigge is used as forecast and compared to the ERA5 reanalysis data.

In [ ]:
target_obs = 'Precipitation_Reanalysis_2006-2018.nc'
DS_observation = xr.open_dataset(target_obs)
target_hres = 'Precipitation_hres_24.nc'
DS_predictor = xr.open_dataset(target_hres)

# index
m = 0
# start dataset from 01.01.2006 
first_day = 5
CPA_All = np.zeros(45)
CPAEurop = np.zeros((199,279))
# indices for reanalysis and hres
indices_start = np.array([0,   90,  181,  273,  365,  456,  547,  639,  731,  821,  912,
       1004, 1096, 1186, 1277, 1369, 1461, 1551, 1642, 1734, 1826, 1917,
       2008, 2100, 2192, 2282, 2373, 2465, 2557, 2647, 2738, 2830, 2922,
       3012, 3103, 3195, 3287, 3378, 3469, 3561, 3653, 3743, 3834, 3926,
       4018])
indices_end = np.array([365,  456,  547,  639,  731,  821,  912, 1004, 1096, 1186, 1277,
       1369, 1461, 1551, 1642, 1734, 1826, 1917, 2008, 2100, 2192, 2282,
       2373, 2465, 2557, 2647, 2738, 2830, 2922, 3012, 3103, 3195, 3287,
       3378, 3469, 3561, 3653, 3743, 3834, 3926, 4018, 4108, 4199, 4291,
       4383])

indices_start_rea = indices_start + first_day
indices_end_rea = indices_end + first_day

for indx in range(0,45):
    start = indices_start[indx]
    end = indices_end[indx]
    start_rea = indices_start_rea[indx]
    end_rea = indices_end_rea[indx]
    
    predictor_nc = np.asarray(DS_predictor.tp[start:end,:,:])
    response_nc = np.asarray(DS_observation.tp[start_rea:end_rea,:,:])

    response = response_nc.flatten()
    predictor = predictor_nc.flatten()
             
    resOrder = np.argsort(response)
    resSort = response[resOrder] 
    preSort = predictor[resOrder]                
    preRank = rankdata(preSort, method='average')
    resRank = rankdata(resSort, method='average')
    resClass = rankdata(resSort, method='dense')
    CPA_All[m] = (np.cov(resClass,preRank)[0][1]/np.cov(resClass,resRank)[0][1]+1)/2 
    print(CPA_All[m])
    m = m+1
print(CPA_All)

np.savetxt('CPA_Precipitation_hres_24.txt',CPA_All)